In [6]:
using LinearAlgebra, Statistics, Distributions, Plots

# Liczba agentów rodzaju I
Nᴵ = 5000
# Liczba agentów rodzaju II
Nᴵᴵ = 25000
# Liczba agentów rodzaju III
Nᴵᴵᴵ = 50000
# Suma 80000 (8*10^4)

# Ceny
P_górna⁽¹⁾ = 35000
P_górna⁽²⁾ = 25000
P_dolna⁽¹⁾ = 23000
P_górna⁽³⁾ = 19000
P_dolna⁽²⁾ = 18000
P_dolna⁽³⁾ = 8000

T = 12 * 20  # 12 miesięcy - rok, 20 lat
CZAS = 15 * 12  # Piętnastoletnie ,,pożyczki"

# Początkowe rozmieszczenie - przestrzenie trzech dzielnic: ciągłe
α₁ = 1
θ₁ = 0.1
α₂ = 2
θ₂ = 0.1
α₃ = 5
θ₃ = 0.1

Pocz_rozk_I = Pareto(α₁, θ₁);
Pocz_rozk_II = Pareto(α₂, θ₂);
Pocz_rozk_III = Pareto(α₃, θ₃);

AGENCI = zeros(Nᴵ + Nᴵᴵ + Nᴵᴵᴵ, 5, T);

In [7]:

# Utwórz pętlę do symulacji
function symulacja(liczba_iteracji)
    wyniki = zeros(3, T, liczba_iteracji)

    for iteracja in 1:liczba_iteracji
        # Inicjalizacja początkowego rozmieszczenia agentów
        for n in 1:Nᴵ
            AGENCI[n, 2, 1] = rand(Pocz_rozk_I)
            AGENCI[n, 1, 1] = 1
            AGENCI[n, 3, 1] = AGENCI[n, 2, 1] 
            AGENCI[n, 4, 1] = rand(Uniform(0, 1))
        end

        for n in Nᴵ+1:Nᴵᴵ
            AGENCI[n, 2, 1] = rand(Pocz_rozk_II)
            AGENCI[n, 1, 1] = 2
            AGENCI[n, 3, 1] = AGENCI[n, 2, 1] 
            AGENCI[n, 4, 1] = rand(Uniform(0, 1))
        end

        for n in Nᴵᴵ+1:Nᴵᴵᴵ
            AGENCI[n, 2, 1] = rand(Pocz_rozk_III)
            AGENCI[n, 1, 1] = 2
            AGENCI[n, 3, 1] = AGENCI[n, 2, 1] 
            AGENCI[n, 4, 1] = rand(Uniform(0, 1))
        end

       
        for t in 1:T
            # Liczba mieszkańców poszczególnych dzielnic w danym okresie
            wyniki[1, t, iteracja] = count(AGENCI[:, 1, t] .== 1)
            wyniki[2, t, iteracja] = count(AGENCI[:, 1, t] .== 2)
            wyniki[3, t, iteracja] = count(AGENCI[:, 1, t] .== 3)

            # Zmiana netto liczby mieszkańców danej dzielnicy w danym okresie - w wartościach procentowych
            if t > 1
                for dzielnica in 1:3
                    zmiana = wyniki[dzielnica, t, iteracja] - wyniki[dzielnica, t-1, iteracja]
                    zmiana_procentowa = (zmiana / wyniki[dzielnica, t-1, iteracja]) * 100
                    wyniki[dzielnica, t, iteracja] = zmiana_procentowa
                end
            end
        end
    end
    return wyniki
end



symulacja (generic function with 1 method)

In [8]:
liczba_iteracji = 100
wyniki_symulacji = symulacja(liczba_iteracji)

# Tworzenie rysunków
function rysuj_statystyki(wyniki)
    for dzielnica in 1:3
        nazwa_dzielnicy = "Dzielnica $dzielnica"
        # Tworzenie wykresu średniej liczby mieszkańców w poszczególnych okresach
        # z zakresem niepewności
        plot(mean(wyniki[dzielnica, :, :], dims=3), ribbon=std(wyniki[dzielnica, :, :], dims=3),
             xlabel="Okres", ylabel="Średnia liczba mieszkańców", title=nazwa_dzielnicy,
             label="", fillalpha=0.3)
    end
end

rysuj_statystyki(wyniki_symulacji)